# Instruction Finetuning using LoRA

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r '/content/drive/MyDrive/Gen AI/Finetuning/finetuning-llm-code/requirements.txt'

In [3]:
!wandb login

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import wandb

if wandb.api.api_key:
    print("WandB login successful")
else:
    print("WandB login failed")

WandB login successful


In [5]:
from huggingface_hub import login

# Replace 'YOUR_HUGGING_FACE_TOKEN' with your actual token
login("")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In this notebook, we will look into how to perform instruction finetuning using LoRA PEFT method. The task is to perform Supervised finetuning (SFT) of OpenHathi model on a small Hingligh instruct dataset.

Load the required libraries

In [6]:
import os
os.environ["WANDB_PROJECT"]="openhathi_instruct_finetuning"

from enum import Enum
from functools import partial
import pandas as pd
import torch

from transformers import AutoModelForCausalLM, LlamaTokenizer, TrainingArguments, set_seed
from datasets import load_dataset
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, TaskType

seed = 42
set_seed(seed)

## Data preprocessing

In [7]:
from transformers import AutoTokenizer

In [10]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [11]:
'<|im_start|>user\nKya Mother Teresa Nobel Peace Prize jeeti thi?<|im_end|>\n<|im_start|>assistant\nHaan, Mother Teresa ne 1979 mein Nobel Peace Prize jeeti thi, unke social work aur seva ke liye.<|im_end|>\n'

'<|im_start|>user\nKya Mother Teresa Nobel Peace Prize jeeti thi?<|im_end|>\n<|im_start|>assistant\nHaan, Mother Teresa ne 1979 mein Nobel Peace Prize jeeti thi, unke social work aur seva ke liye.<|im_end|>\n'

In [12]:
from datasets import load_dataset

# Define the chat template format
template = """{% for message in messages %}
{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}
{% if loop.last and add_generation_prompt %}
{{'<|im_start|>assistant\n'}}
{% endif %}
{% endfor %}
"""

def preprocess(samples):
    batch = []
    # Iterate over each row of the dataset
    for i in range(len(samples["input"])):
        # Extract instruction, input (text), and output (label)
        instruction = samples["instruction"][i]
        input_text = samples["input"][i]
        output_text = samples["output"][i]

        # Format as per the required template
        conversation = {
            'role': 'user',
            'content': f"{instruction} {input_text}"
        }
        response = {
            'role': 'assistant',
            'content': output_text
        }

        # Use the predefined template to create the content
        formatted_conversation = (
            f"<|im_start|>user\n{conversation['content']}<|im_end|>\n"
            f"<|im_start|>assistant\n{response['content']}<|im_end|>\n"
        )

        # Append to batch
        batch.append(formatted_conversation)

    # Return processed batch
    return {"content": batch}

# Load the dataset
dataset_name = "FinGPT/fingpt-sentiment-train"  # Use your dataset path
dataset = load_dataset(dataset_name)

# Reduce the size of the 'train' split to 1000 rows
dataset['train'] = dataset['train'].shuffle(seed=42).select(range(1000))

# Apply the preprocess function to each batch of the dataset
dataset = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# Split the dataset into train and test sets
dataset = dataset["train"].train_test_split(test_size=0.1)

# Check a sample from the processed dataset
print(dataset["train"][0])

README.md:   0%|          | 0.00/529 [00:00<?, ?B/s]

(…)-00000-of-00001-dabab110260ac909.parquet:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/76772 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'content': '<|im_start|>user\nWhat is the sentiment of this news? Please choose an answer from {negative/neutral/positive}. Via the move , the company aims annual savings of some EUR 3 million USD 4.3 m , the main part of which are expected to be realized this year .<|im_end|>\n<|im_start|>assistant\npositive<|im_end|>\n'}


In [13]:
print(dataset["train"][0])

{'content': '<|im_start|>user\nWhat is the sentiment of this news? Please choose an answer from {negative/neutral/positive}. Via the move , the company aims annual savings of some EUR 3 million USD 4.3 m , the main part of which are expected to be realized this year .<|im_end|>\n<|im_start|>assistant\npositive<|im_end|>\n'}


In [14]:
print(dataset["test"][0])


{'content': '<|im_start|>user\nWhat is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}. "Frozen II" is expected to heat up a cold box office, with $130 million in sales expected over its opening weekend https://t.co/jVip0q50Ne<|im_end|>\n<|im_start|>assistant\nneutral<|im_end|>\n'}


## Create the PEFT model

### LoRA Config

In [18]:
peft_config = LoraConfig(r=8,
                         lora_alpha=16,
                         lora_dropout=0.1,
                         target_modules=["gate_proj","q_proj","lm_head","o_proj","k_proj","embed_tokens","down_proj","up_proj","v_proj"],
                         task_type=TaskType.CAUSAL_LM)

In [19]:
class ChatmlSpecialTokens(str, Enum):
    user = "<|im_start|>user"
    assistant = "<|im_start|>assistant"
    system = "<|im_start|>system"
    eos_token = "<|im_end|>"
    bos_token = "<s>"
    pad_token = "<pad>"

    @classmethod
    def list(cls):
        return [c.value for c in cls]

tokenizer = LlamaTokenizer.from_pretrained(
        model_name,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        bos_token=ChatmlSpecialTokens.bos_token.value,
        eos_token=ChatmlSpecialTokens.eos_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list(),
        trust_remote_code=True
    )
tokenizer.chat_template = template
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# cast non-trainable params in fp16
for p in model.parameters():
    if not p.requires_grad:
        p.data = p.to(torch.float16)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


trainable params: 21,549,136 || all params: 7,263,322,192 || trainable%: 0.2967


In [20]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): lora.Embedding(
          (base_layer): Embedding(32005, 4096)
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.1, inplace=False)
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 8x32005])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 4096x8])
          (lora_magnitude_vector): ModuleDict()
        )
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=Fals

## Training

In [21]:
output_dir = "fingpt_instruct"
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 8
logging_steps = 5
learning_rate = 5e-4
max_grad_norm = 1.0
max_steps = 250
num_train_epochs=10
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_seq_length = 2048

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_strategy="no",
    evaluation_strategy="epoch",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    fp16=True,
    report_to=["tensorboard", "wandb"],
    hub_private_repo=True,
    push_to_hub=True,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False}
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    packing=True,
    dataset_text_field="content",
    max_seq_length=max_seq_length,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': packing, dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:213: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:301: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will overr

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
trainer.train()
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pratikpophali113 (pratikpophali113-self). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,No log,2.754318
1,2.718100,2.602391
2,2.552400,2.227631
4,2.050500,1.663988
5,1.370900,1.561388
6,0.861100,1.635600
8,0.653800,1.691900


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:228: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/611M [00:00<?, ?B/s]

events.out.tfevents.1729101391.d4ccc0588b4a.2005.0:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
!nvidia-smi

Wed Oct 16 18:10:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   71C    P0              33W /  72W |  16955MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Loading the trained model and getting the predictions of the trained model

In [8]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch

peft_model_id = "Pratik3194/fingpt_instruct"
device = "cuda"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/797 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


adapter_model.safetensors:   0%|          | 0.00/611M [00:00<?, ?B/s]

In [9]:
# Diye gaye sentence ko English mein translate kare:\n  Maine 10 baar customer care ko call kiya hai par abhi tak mujhe mera order nahi mila hai.
# bro, ye generative AI kya hai?
# भारत में पर्यटन के बारे में एक निबंध लिखें।

model.to(torch.float16)
model.cuda()
model.eval()
messages = [
    {"role": "user", "content": "What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}. 'Frozen II' is expected to heat up a cold box office, with $130 million in sales expected over its opening weekend https://t.co/jVip0q50Ne"},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")#, add_special_tokens=False)
inputs = {k: v.to("cuda") for k,v in inputs.items()}
outputs = model.generate(**inputs,
                         max_new_tokens=128,
                         do_sample=True,
                         top_p=0.95,
                         temperature=0.2,
                         repetition_penalty=1.1,
                         eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s><|im_start|>user
What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}. 'Frozen II' is expected to heat up a cold box office, with $130 million in sales expected over its opening weekend https://t.co/jVip0q50Ne<|im_end|>

<|im_start|>assistant

neutral<|im_end|>


In [10]:
!nvidia-smi

Wed Oct 16 18:21:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   58C    P0              30W /  72W |  14269MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--